In [1]:
%matplotlib inline

import numpy as np
import cgs as cgs

import df as df
import dm_density_profiles as dm
import particle_distribution as pd

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt


using cgs py


In [2]:
NFW = dm.general_dm_profile('NFW')
NFW.set_params(profile_shape_params=[1.0,3.0,1.0])
NFW.set_params(M_vir = 1.0E12 * cgs.Msun, r_vir = 240.0 * cgs.kpc)
NFW.set_params(r_decay = 0.1*NFW.r_vir, r_s = (1.0/12.0)*NFW.r_vir)

df_filename = "./NFW_500.dat"

NFW_DF = df.DF(NFW)
f = NFW_DF.load_df(df_filename)


 500 DF points successfuly loaded from ./NFW_500.dat


In [3]:
NFW_PD = pd.particle_distribution(NFW_DF,1.0)

In [ ]:
pos, vel = NFW_PD.generate_particle_distribution(1.0E4)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111,projection='3d')
pos = pos/cgs.kpc
ax.scatter(pos[:,0], pos[:,1], pos[:,2])


In [ ]:
r = np.sqrt(NFW_PD.pos[:,0]**2 + NFW_PD.pos[:,1]**2 + NFW_PD.pos[:,2]**2)    

r_bins = np.linspace(0.0, np.max(r), nbins + 1)
# now bin with np hist
r_hist, r_bins = np.histogram(r, bins = r_bins)

# calculate the volume in each bin
volume = 4.0 * np.pi * (r_bins[1:]**3 - r_bins[:-1]**3) / 3.0

# now calculate the bin centers
r_cent = 0.5*(r_bins[1:] + r_bins[:-1])

# number density
density = r_hist / volume


In [ ]:
nbins = 100
#density = NFW_PD.density_profile(nbins)

fig = plt.figure(figsize=(14,6))
ax1 = fig.add_subplot(121)


mass_density = density * (NFW_PD.DF.dprof.M_sys/NFW_PD.N_part)
analytic_density =  NFW_PD.DF.dprof.density(r_cent)


ax1.plot(r_cent/cgs.kpc, analytic_density, lw = 2.5, label = 'Analytic Profile')
ax1.plot(r_cent/cgs.kpc, mass_density, lw = 2.5, label = 'Particle Distribution')
ax1.semilogy()
ax1.set_ylabel(r'Density (g/cm$^{-3}$)')
ax1.set_xlabel(r'r (kpc)')
ax1.legend(loc='best')


error = (analytic_density - mass_density)/analytic_density
error = np.abs(error)


ax2 = fig.add_subplot(122)
ax2.plot(r_cent/cgs.kpc, error, lw=2.5)
ax2.set_xlabel(r'r (kpc)')
ax2.set_ylabel(r'Absolute Error')
